In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
from pathlib import Path

import pytorch_lightning as pl

import torch
from torch import nn
from torch_geometric.nn import knn_graph
from torch_geometric import transforms
from torch_geometric.data import Data, DataLoader
from torchmetrics.functional import mean_squared_error

from gvp import SyntheticGNN, SyntheticGVP, SyntheticDataModule

In [11]:
data_dir = Path("../data/synthetic")

In [14]:
model = SyntheticGVP(1, 1, 1, 1, 20, 4)

transform = transforms.Compose([
    transforms.KNNGraph(k=10),
    transforms.Cartesian(),
    transforms.Distance(),
])

dm = SyntheticDataModule(data_dir, 32, "off_center", transform, num_workers=2)
dm.setup('fit')

# data0 = next(iter(dm.train_dataloader()))
# data0.x

In [15]:
for batch in dm.train_dataloader():
    out = model(batch).view(-1)
    loss = mean_squared_error(out, batch.y)
    print(loss)
    break

TypeError: forward() takes 2 positional arguments but 3 were given